In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams

rcParams["savefig.dpi"] = 200
rcParams["font.size"] = 8

import warnings

warnings.filterwarnings("ignore")

# Create 3D region masks

In this tutorial we will show how to create 3D boolean masks for arbitrary latitude and longitude grids. It uses the same algorithm to determine if a gridpoint is in a region as for the 2D mask. However, it returns a `xarray.Dataset` with shape `region x lat x lon`, gridpoints that do not fall in a region are `False`, the gridpoints that fall in a region are `True`.

Import regionmask and check the version:

In [ ]:
import regionmask

regionmask.__version__

Load xarray and numpy:

In [ ]:
import xarray as xr
import numpy as np

## Creating a mask

We define a lon/ lat grid with a 1° grid spacing, where the points define the center of the grid.

In [ ]:
lon = np.arange(-179.5, 180)
lat = np.arange(-89.5, 90)

We create a mask with the SREX regions (Seneviratne et al., 2012).

In [ ]:
regionmask.defined_regions.srex

The function `mask_3D` determines which gripoints lie within the polygon making up the each region:

In [ ]:
mask = regionmask.defined_regions.srex.mask_3D(lon, lat)
mask

As mentioned, `mask` is a boolean `xarray.Dataset` with shape `region x lat x lon`. It contains `region` as dimension coordinate and `abbrev` and `name` as non-dimension coordinates (see the xarray docs for the details on the [terminology](http://xarray.pydata.org/en/stable/terminology.html)). The four first layers look as follows:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import colors as mplc

cmap1 = mplc.ListedColormap(["none", "#9ecae1"])

fg = mask.isel(region=slice(4)).plot(
    subplot_kws=dict(projection=ccrs.PlateCarree()),
    col="region",
    col_wrap=2,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
    aspect=1.5,
    cmap=cmap1,
)

for ax in fg.axes.flatten():
    ax.coastlines()

fg.fig.subplots_adjust(hspace=0, wspace=0.1);

## Working with a 3D mask

masks can be used to select data in a certain region and to calculate regional averages - let's illustrate this with a 'real' dataset:

In [ ]:
airtemps = xr.tutorial.load_dataset("air_temperature")

The example data is a temperature field over North America. Let's plot the first time step:

In [ ]:
# choose a good projection for regional maps
proj = ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

airtemps.isel(time=1).air.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

An xarray object can be passed to the `mask_3D` function:

In [ ]:
mask_full = regionmask.defined_regions.srex.mask_3D(airtemps)
mask_full

Per default this creates a `mask` containing one layer for each individual region - even if no gridpoint is contained in this region (all points are `False`). As the example data only has values over Northern America most layers will only contain `False`. To only obtain those layers containing (at least) one gridpoint specify `drop=True`:

In [ ]:
mask_drop = regionmask.defined_regions.srex.mask_3D(airtemps, drop=True)
mask_drop

Note how there are only 6 layers.

### Calculate weighted regional averages

Using the 3-dimensional mask it is possible to calculate weighted averages of all regions in one go, using the `weighted` method (requires xarray 0.15.1 or later). As proxy of the grid cell area we use `cos(lat)`:

In [ ]:
weights = np.cos(np.deg2rad(airtemps.lat))

ts_airtemps_regional = airtemps.weighted(mask_drop * weights).mean(dim=("lat", "lon"))
ts_airtemps_regional

The regionally-averaged time series can be plotted:

In [ ]:
ts_airtemps_regional.air.plot(col="region", col_wrap=3);

### Restrict the mask to land points

Combining the mask of the regions with a land-sea mask we can create a land-only mask. We use the `natural_earth.land_110` regions to create a land-sea mask:

In [ ]:
land_110 = regionmask.defined_regions.natural_earth.land_110

land_sea_mask = land_110.mask_3D(airtemps)

and plot it

In [ ]:
proj = ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

land_sea_mask.squeeze().plot.pcolormesh(
    ax=ax, transform=ccrs.PlateCarree(), cmap=cmap1, add_colorbar=False
)

ax.coastlines();

To create the combined mask we multiply the two:

In [ ]:
mask_lsm = mask_drop * land_sea_mask.squeeze(drop=True)

Note the `squeeze(drop=True)`. This is required to remove the regions dimension from `land_sea_mask`.

Finally we compare the original mask with the one restricted to land points:

In [ ]:
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=proj))


ax = axes[0]
mask_drop.sel(region=2).plot(
    ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap=cmap1
)
ax.coastlines()
ax.set_title("Regional mask: all points")

ax = axes[1]
mask_lsm.sel(region=2).plot(
    ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap=cmap1
)
ax.coastlines()
ax.set_title("Regional mask: land only");

## References

* Special Report on Managing the Risks of Extreme Events and Disasters to Advance Climate Change Adaptation (SREX, Seneviratne et al., 2012: https://www.ipcc.ch/site/assets/uploads/2018/03/SREX-Ch3-Supplement_FINAL-1.pdf)